## Setup

In [1]:
!nvidia-smi

Fri Feb 14 05:30:08 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:0A:00.0 Off |                    0 |
| N/A   29C    P0              51W / 500W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 132.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 143.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.9/686.9 KB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 99.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 15.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 15.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 12.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()
@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

In [4]:
%load_ext dotenv
%dotenv

## Configuration

In [5]:
import os

In [8]:
## CONFIG
NUM_GPUS = 4
HF_TOKEN = os.environ["HF_TOKEN_R"]
IGNORE_PATTERNS = "original/consolidated*"
CONFIG_FILE = "llama_3_1_8b_lora_distributed.yaml"

## MODEL
FT_MODEL_REPO = "multimodalai"
BASE_MODEL_HF_ID = "meta-llama/Llama-3.1-8B"
CLIENT = "resume-critique"
MODEL = "llama3_1_8b"
MODEL_NUMBER = "4"
REV_N = "2"
FT_METHOD = "tt_lora"
MODEL_TYPE = "adapter"

MDATA_ID = f"model_{MODEL_NUMBER}_2k"
REV = f"rev_{REV_N}"
FT_MODEL_NAME = f"{CLIENT}-{MODEL}-{FT_METHOD}-{MDATA_ID}-{MODEL_TYPE}-{REV}"
FT_MODEL_HF_ID = f"multimodalai/{FT_MODEL_NAME}"

## DATASET
TRAINING_DATA = "resume_critique_model_4_v2.jsonl"

## PATH
BASE_MODEL_PATH = "base_model/"
TOKENIZER_PATH = f"{BASE_MODEL_PATH}/original/tokenizer.model"
OUTPUT_MODEL_PATH = f"checkpoint/{FT_MODEL_REPO}/{FT_MODEL_NAME}"
TRAINING_DATA_PATH = f"data/{TRAINING_DATA}"
CONFIG_FILE_PATH = f"config/{CONFIG_FILE}"

## TRACKING
WANDB_GROUP_NAME = CLIENT
RUN_WANDB_NAME = f"run-{FT_MODEL_NAME}"
LOGS_PATH = "logs/"

In [9]:
!mkdir -p {OUTPUT_MODEL_PATH}
!mkdir -p {LOGS_PATH}

## Download Base Model

In [10]:
%%pybash
tune download {BASE_MODEL_HF_ID} --output-dir {BASE_MODEL_PATH} --ignore-patterns {IGNORE_PATTERNS} --hf-token {HF_TOKEN}

Ignoring files matching the following patterns: original/consolidated*


Fetching 16 files: 100%|██████████| 16/16 [01:58<00:00,  7.41s/it]


Successfully downloaded model repo and wrote to the following locations:
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/special_tokens_map.json
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/.gitattributes
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/tokenizer.json
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/LICENSE
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/model.safetensors.index.json
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/tokenizer_config.json
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/model-00004-of-00004.safetensors
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/README.md
/home/ubuntu/internal-foundry-fine-tune/resume-critique/model_4_rev_2/base_model/original
/home/ubuntu/internal-foundry-fin

## Fine-Tune

In [ ]:
%%pybash
tune run \
    --nproc_per_node {NUM_GPUS} \
    lora_finetune_distributed \
    --config {CONFIG_FILE_PATH} \
    tokenizer.path={TOKENIZER_PATH} \
    checkpointer.checkpoint_dir={BASE_MODEL_PATH} \
    checkpointer.output_dir={OUTPUT_MODEL_PATH} \
    dataset.data_files={TRAINING_DATA_PATH} \
    metric_logger.group={WANDB_GROUP_NAME} \
    metric_logger.name={RUN_WANDB_NAME} \
	output_dir={OUTPUT_MODEL_PATH} \
	metric_logger.log_dir={LOGS_PATH}

Running with torchrun...


W0214 05:37:43.812000 12629 torch/distributed/run.py:793] 
W0214 05:37:43.812000 12629 torch/distributed/run.py:793] *****************************************
W0214 05:37:43.812000 12629 torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0214 05:37:43.812000 12629 torch/distributed/run.py:793] *****************************************
INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeDistributed with resolved config:

batch_size: 8
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: base_model/
  checkpoint_files:
  - model-00001-of-00004.safetensors
  - model-00002-of-00004.safetensors
  - model-00003-of-00004.safetensors
  - model-00004-of-00004.safetensors
  model_type: LLAMA3
  output_dir: checkpoint/multimodalai/resume-critique-llama3_1_8b-tt_lora-mod

## Save the Model

In [ ]:
## CONFIG
NUM_GPUS = 4
HF_TOKEN = os.environ["HF_TOKEN_R"]
IGNORE_PATTERNS = "original/consolidated*"
CONFIG_FILE = "llama_3_1_8b_lora_distributed.yaml"

## MODEL
ORGANIZATION = "multimodalai"
BASE_MODEL_HF_ID = "meta-llama/Llama-3.1-8B"
CLIENT = "resume-critique"
MODEL = "llama3_1_8b"
MODEL_NUMBER = "4"
REV_N = "2"
FT_METHOD = "tt_lora"
MODEL_TYPE = "adapter"

MDATA_ID = f"model_{MODEL_NUMBER}_2k"
REV = f"rev_{REV_N}"
FT_MODEL_NAME = f"{CLIENT}-{MODEL}-{FT_METHOD}-{MDATA_ID}-{MODEL_TYPE}-{REV}"
FT_MODEL_HF_ID = f"multimodalai/{FT_MODEL_NAME}"

## DATASET
TRAINING_DATA = "resume_critique_model_4_v2.jsonl"

## PATH
BASE_MODEL_PATH = "base_model/"
TOKENIZER_PATH = f"{BASE_MODEL_PATH}/original/tokenizer.model"
OUTPUT_MODEL_PATH = f"checkpoint/{ORGANIZATION}/{FT_MODEL_NAME}"
TRAINING_DATA_PATH = f"data/{TRAINING_DATA}"
CONFIG_FILE_PATH = f"config/{CONFIG_FILE}"

## TRACKING
WANDB_GROUP_NAME = CLIENT
RUN_WANDB_NAME = f"run-{FT_MODEL_NAME}"
LOGS_PATH = "logs/"

In [ ]:
HF_TOKEN = os.environ["HF_TOKEN_W"]

In [ ]:
%%pybash
huggingface-cli login --token {HF_TOKEN}
huggingface-cli repo create -y --organization {ORGANIZATION} {FT_MODEL_NAME}
huggingface-cli upload {FT_MODEL_HF_ID} {OUTPUT_MODEL_PATH}